# Environment


In [1]:
# basci env
import os
import pandas as pd
import time
import json
from tqdm import tqdm

# data process of file from ncbi
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqUtils import MeltingTemp as mt

# # get gene data from ncbi
# from Bio import Entrez

# # blast and xml file process
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

In [178]:
# dir
workdir = './dataset/2024.1.23_Sindy_Isoforms_lncRNA/'
os.makedirs(workdir, exist_ok=True)

current_time = time.localtime()
formatted_time = time.strftime("%Y%m%d_%H%M%S", current_time)

output = os.path.join(workdir, 'results', formatted_time)
pre_binding_dir = os.path.join(output, "pre_binding")
os.makedirs(output, exist_ok=True)

# basic variables
gene_name_list_tosearch = "gene_name_list_tosearch.txt"
pre_binding_file_suffix = "_pre_binding.fasta"
total_pre_binding_file_name = "_total.fasta"

# tmp file
gene_name_list_file = "lncRNA_isoforms.xlsx"
pre_binding_num_file = "pre_binding_num.json"
blast_results_file = "blast_results.xml"

In [183]:
organism = 'mouse'
gene_info = pd.read_excel(os.path.join(workdir, gene_name_list_file), sheet_name='Sheet1')
# gene_list = list(gene_info['gene_name'].unique())
id_list = list(gene_info['ensembl_id'])
isoform_list = list(gene_info['isoform'])
gene_list = [_.split('-')[0] for _ in isoform_list]

# Binding site generate

## Example usage


### get seq from ensembl

In [4]:
# # example usage of requests on ensembl
# # check lib.database_interaction for ensembl_id_to_seqs and ensembl_name_to_seqs for more details
# import requests

# species = "human"
# gene = "BRCA1"
# seq_type = "cds"

# def get_seqs(gene, species, seq_type):
#     lookup_url = f"http://rest.ensembl.org/lookup/symbol/{species}/{gene}?content-type=application/json"
#     gene_id = requests.get(url=lookup_url).json()["id"]

#     transcripts_url = f"http://rest.ensembl.org/overlap/id/{gene_id}?feature=transcript;content-type=application/json"
#     transcripts = requests.get(url=transcripts_url).json()

#     # Get sequences for each transcript
#     sequences = {}
#     for transcript in tqdm(transcripts, desc=f'Gene_{gene}'):
#         try:
#             seq_name = "_".join([transcript["id"], transcript["external_name"], transcript['biotype']])
#             seq_url = f"http://rest.ensembl.org/sequence/id/{transcript['id']}?type={seq_type};content-type=application/json"
#             seq_response = requests.get(seq_url).json()
#             sequences[seq_name] = seq_response["seq"]
#         except: continue

#     return sequences

# sequences = get_seqs(gene, species, seq_type)    

### find_most_distinctive_substrings

In [5]:
# from lib.search_binding import step_by_step


# def calculate_optimal_alignment_similarity(sub1, sub2):
#     max_similarity = 0
#     for shift in range(-len(sub1) + 1, len(sub1)):
#         shifted_similarity = sum((sub1[i] == sub2[i - shift]) for i in range(len(sub1)) if 0 <= i - shift < len(sub2))
#         max_similarity = max(max_similarity, shifted_similarity)
#     return max_similarity

# def find_most_distinctive_substrings(sequences_dict, substring_length=40, top_n=10):
#     if any(len(seq) < substring_length for seq in sequences_dict.values()):
#         raise ValueError("One or more sequences are shorter than the specified substring length.")
    
#     all_substrings = {}
#     for name, seq in sequences_dict.items():
#         _, _, selected_substrings, _ = step_by_step(
#         seq,
#         BDS_len=40,
#         BDS_num=50,
#         min_gap=1,
#         better_gap=40,
#         gene=name,
#         G_min=0.25,
#         G_max=0.7,
#         G_consecutive=5,
#         Tm_low=50,
#         Tm_high=65,
#         show_process=False, 
#         warn=False
#         )
        
#         all_substrings[name] = selected_substrings

#     top_substrings = {}
#     for name, substrings in all_substrings.items():
#         distinctiveness_scores = {}
#         for sub in tqdm(substrings, desc=name):
#             distinctiveness_scores[sub] = min(calculate_optimal_alignment_similarity(sub, other_sub)
#                                               for other_name, other_substrings in all_substrings.items()
#                                               if other_name != name
#                                               for other_sub in other_substrings)

#         top_n_substrings = sorted(distinctiveness_scores, key=distinctiveness_scores.get)[:top_n]
#         top_substrings[name] = top_n_substrings

#     return top_substrings

# distinctive_substrings = find_most_distinctive_substrings(sequences, substring_length=40)


## Get sequences from ensembl dataset


In [185]:
from lib.database_interaction import ensembl_name_to_seqs


sequences_of_all = dict()
skip, trial = 0, 0
while True:
    if skip == len(gene_list): break
    if trial > 3:
        skip += 1
        trial = 0
    try:
        tmp_gene_list = gene_list[skip:]
        tmp_id_list = id_list[skip:]
        for id, gene in zip(tmp_id_list, tmp_gene_list):
            sequences_of_all[gene] = ensembl_name_to_seqs(gene=gene, species='mouse', seq_type='')
            skip += 1
    except: 
        trial += 1
        time.sleep(3)

Gene:	2900045O20Rik: 100%|██████████| 1/1 [00:00<00:00,  2.04it/s]


In [186]:
with open(os.path.join(output, 'sequence_of_all.json'), 'w') as file: json.dump(sequences_of_all, file)

### check

In [188]:
nothing_found = []
isoform_not_found = []
sequences_of_all_filtered = dict()

for gene, isoforms in sequences_of_all.items():
    sequences_of_all_filtered[gene] = []
    for isoform in isoforms:
        if gene in isoform['external_name']:
            sequences_of_all_filtered[gene].append(isoform)

all_isoform = []
for gene, isoforms in sequences_of_all_filtered.items():
    if len(isoforms) < 1:
        nothing_found.append(gene)
    all_isoform += [_['external_name'] for _ in isoforms]

isoform_not_found = [_ for _ in isoform_list if _ not in all_isoform]
print('Genes_not_found: {}'.format(', '.join(nothing_found)))
print('Isoforms_not_found: {}'.format(', '.join(isoform_not_found)))

Genes_not_found: 
Isoforms_not_found: Gm11032-201, Gm10563-201, AC158554.1-201, Gm17555-201, AC125141.1-201, Gm21750-201, A230050P20Rik-204, Gm17571-201, Gm26945-201, Gm42418-201, Gm26917-202, AY036118-201


In [202]:
isoform_not_found

['Gm11032-201',
 'Gm10563-201',
 'AC158554.1-201',
 'Gm17555-201',
 'AC125141.1-201',
 'Gm21750-201',
 'A230050P20Rik-204',
 'Gm17571-201',
 'Gm26945-201',
 'Gm42418-201',
 'Gm26917-202',
 'AY036118-201']

## Find potential bds for interested isoform

In [192]:
isoform_interested = []
for gene, isoforms in sequences_of_all.items():
    for isoform in isoforms:
        if isoform['external_name'] in isoform_list:
            isoform['parent_name'] = gene
            isoform_interested.append(isoform)

In [193]:
from lib.search_binding import step_by_step


for isoform in tqdm(isoform_interested, 'Find potential bds for isoforms'):
    Tm_l, Tm_r, selected_substrings, positions = step_by_step(
    sequence=isoform['seq'], gene=isoform['parent_name'],
    BDS_len=40, BDS_num=50, min_gap=1, better_gap=40,
    G_min=0.25, G_max=0.7, G_consecutive=5, Tm_low=50, Tm_high=65,
    show_process=False,  warn=False)
    
    isoform['potential_bds'] = []
    for Tm_l_one, Tm_r_one, substring, position in zip(Tm_l, Tm_r, selected_substrings, positions):
        isoform['potential_bds'].append({
            'seq': substring, 'position': position,
             
            'Tm_l': Tm_l_one, 'Tm_r': Tm_r_one})

Find potential bds for isoforms: 100%|██████████| 25/25 [00:03<00:00,  6.54it/s]


## Find specific binding site for each isoform

In [194]:
from lib.search_binding import seq_minus

def calculate_optimal_alignment_similarity(sub1, sub2):
    max_similarity = 0
    for shift in range(-len(sub1) + 1, len(sub1)):
        shifted_similarity = sum((sub1[i] == sub2[i - shift]) for i in range(len(sub1)) if 0 <= i - shift < len(sub2))
        max_similarity = max(max_similarity, shifted_similarity)
    return max_similarity

In [195]:
for isoform in tqdm(isoform_interested, desc='Calculating similarity'):
    isoform_name = isoform['external_name']
    parent_name = isoform['parent_name']
    ref_isoforms = [isoform_ref['seq'] for isoform_ref in sequences_of_all[parent_name] if isoform_ref['external_name'] != isoform_name]
    for potential_bds in isoform['potential_bds']:
        try: potential_bds['similarity'] = max([calculate_optimal_alignment_similarity(potential_bds['seq'], ref_seq) for ref_seq in ref_isoforms])
        except ValueError: potential_bds['similarity'] = 0
        potential_bds['seq'] = seq_minus(potential_bds['seq'])

with open(os.path.join(output, 'isoform_interested.json'), 'w') as file: json.dump(isoform_interested, file)

Calculating similarity: 100%|██████████| 25/25 [00:03<00:00,  7.55it/s]


## Save binding site to fasta file

In [196]:
# Search binding sites on mRNA sequence
os.makedirs(os.path.join(output, 'potential_bds'), exist_ok=True)
with open(os.path.join(output, "potential_bds.fasta"), "w") as handle: handle.write("")

for isoform in isoform_interested:
    record_list = []
    for potential_bds in isoform['potential_bds']:
        record_list.append(SeqRecord(Seq(potential_bds['seq']), id=isoform['id'], name=isoform['external_name'], 
                                     description='|'.join([isoform['feature_type'], isoform['biotype']])))

    # write pre_binding to files
    with open(os.path.join(output, 'potential_bds', f"{isoform['external_name']}_potential_bds.fasta"), "w") as f:
        for new_record in record_list:
            SeqIO.write(new_record, f, "fasta")
            
    with open(os.path.join(output, "potential_bds.fasta"), "a") as handle:
        for new_record in record_list:
            SeqIO.write(new_record, handle, "fasta")

# Blast and extract blast results

In [ ]:
# with open(file_out_dir + total_pre_binding_file_name, "r") as f:
#     fasta_string = f.read()
# txid = [2697049]  # organism

# # Submit BLAST search and get handle object
# handle = NCBIWWW.qblast(
#     program="blastn",
#     megablast="yes",
#     database="refseq_rna",
#     sequence=fasta_string,
#     url_base="https://blast.ncbi.nlm.nih.gov/Blast.cgi",
#     format_object="Alignment",
#     format_type="Xml",
# )

# # read handle object and save to a file
# with open(tmp + blast_results_file, "w") as f:
#     f.write(handle.read())

In [197]:
from Bio.Blast import NCBIXML


def extract_blast_record(bds_info, blast_record):
    length = len(blast_record.alignments)
    bds_info["align_num"] = length
    bds_info['descrip'] = dict()
    for i in range(length):
        pm = blast_record.alignments[i].hsps[0].frame[1]
        bds_info["descrip"][i+1] = (blast_record.descriptions[i].title + f"|pm={pm}")

In [199]:
with open(os.path.join(output, 'blast_results.xml'), "r") as blast_output:
    blast_records = NCBIXML.parse(blast_output)
    blast_records_list = [_ for _ in blast_records]
    
    skip = 0
    for _, isoform in enumerate(isoform_interested):    
        bds_num = len(isoform['potential_bds'])
        for bds_order in range(bds_num):
            blast_record = blast_records_list[skip+bds_order]
            extract_blast_record(isoform['potential_bds'][bds_order], blast_record)
        skip += bds_num

# Filter wanted binding site


In [200]:
import re
from copy import deepcopy

pattern = r"transcript variant (\w+)"
def filter_bds_specificity(bds_info, gene):
    align = bds_info['descrip']
    variant = []
    for item in align.values():
        variant += re.findall(pattern, item)
        if gene.upper() not in item.upper():
            return False
    if len(set(variant)) > 1: return False
    else: return True

isoform_interested_select = deepcopy(isoform_interested)
for isoform in isoform_interested_select:
    gene = isoform['parent_name']
    isoform['potential_bds'] = [item for item in isoform['potential_bds'] 
                                if filter_bds_specificity(item, gene)]

with open(os.path.join(output, 'isoform_interested_specific.json'), 'w') as file: json.dump(isoform_interested_select, file)

# Export as xlsx

In [201]:
from lib.search_binding import find_max_min_difference_fixed_length_subsequence as find_pos


bds_df = pd.DataFrame(columns=['accession', 'gene_name', 'mol_type', 'organism',
                            'pos_on_seq', 'binding', 'Tm_l', 'Tm_r', 'wanted', 'align_num', 'align_accession', 'align_descrip', 'plus/minus'])
for isoform in isoform_interested_select:
    bds_position = [potential_bds['position'] for potential_bds in isoform['potential_bds']]
    select_position = find_pos(arr=bds_position, length=3, min_gap=40)
    for potential_bds in [_ for _ in isoform['potential_bds'] if _['position'] in select_position]:
        add = pd.DataFrame({
            'accession': [isoform['id']], 'gene_name': [isoform['external_name']], 'mol_type': [isoform['feature_type']], 'organism': ['mouse'],
            'pos_on_seq': [potential_bds['position']], 'binding': [potential_bds['seq']], 'Tm_l': [potential_bds['Tm_l']], 'Tm_r': [potential_bds['Tm_r']], 'wanted': [True], 
            'align_num': [potential_bds['align_num']], 
            'align_accession': ["|".join([_.split('|')[3] for _ in potential_bds['descrip'].values()])], 
            'align_descrip': ["|".join([_.split('|')[4] for _ in potential_bds['descrip'].values()])], 
            'plus/minus': ["|".join([_.split('|')[5] for _ in potential_bds['descrip'].values()])],
        })
        bds_df = pd.concat([bds_df, add])
bds_df.index = [_ for _ in range(len(bds_df))]
bds_df.to_excel(os.path.join(output, 'probes_wanted.xlsx'))

Gene : 	No valid positions, please loosen your threshold conditions.
Gene : 	Not enough pos for 3 binding sites.
Gene : 	condition too harsh, loose to get better results
[539, 540, 541]
Gene : 	No valid positions, please loosen your threshold conditions.
Gene : 	Not enough pos for 3 binding sites.
Gene : 	condition too harsh, loose to get better results
[41, 44, 53, 54, 55, 56]
Gene : 	Not enough pos for 3 binding sites.
Gene : 	condition too harsh, loose to get better results
[135, 139, 144, 147, 631, 634]
Gene : 	No valid positions, please loosen your threshold conditions.
Gene : 	Not enough pos for 3 binding sites.
Gene : 	condition too harsh, loose to get better results
[169]


C:\Users\Mingchuan\AppData\Local\Temp\ipykernel_31792\2724541571.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  bds_df = pd.concat([bds_df, add])
